## Bank Account Management

In [10]:
# Use http request to convert currency
import requests

api = 'http://free.currencyconverterapi.com/api/v5/convert'
query = '?q=EUR_USD&compact=ultra'

r = requests.get(api + query)

data = r.json()
data['EUR_USD']

1.140895

In [27]:
# Write/Read with json file
import json
with open('account.json', 'w') as json_file:
    json.dump(data, json_file)

In [28]:
with open('account.json') as json_file:  
    test = json.load(json_file)
test

{'EUR_USD': 1.140088}

In [52]:
import requests
import json
from decimal import Decimal
import time
import datetime
from pathlib import Path

class BankAccount:
    def __init__(self, name, balance=0.00):
        file = Path("./account_of_"+ name +".json")
        if file.is_file():
            with open('account_of_'+ name +'.json') as json_file:  
                data = json.load(json_file)
            self.name = data['name']
            self._balance = data['balance']
            self.history = data['history']
            
        else:
            self.name = name
            self._balance = balance
            self.history = []
        self.save()

    def deposit(self, amount, description):
        if isinstance(amount, (int, float)) == False:
            raise TypeError("The inserted amount is not numeric")
        self._balance += amount
        self.history.append('Income of an amount of {}$ at date {} : {}'.format(amount, datetime.datetime.now().date(), description))
        self.save()

    def withdraw(self, amount, description):
        if isinstance(amount, (int, float)) == False:
            raise TypeError("The inserted amount is not numeric")
        if amount > self._balance:
            raise ValueError("Insufficient funds")
        self._balance -= amount
        self.history.append('Withdraw of an amount of {}$ at date {} : {}'.format(amount, datetime.datetime.now().date(), description))
        self.save()
        
    def convert_then_withdraw(self, amount, currency, description):
        if isinstance(amount, (int, float)) == False:
            raise TypeError("The inserted amount is not numeric")
        
        exchange = currency + '_USD'
        api = 'https://free.currencyconverterapi.com/api/v5/convert'
        query = '?q=' + exchange + '&compact=ultra'

        response = requests.get(api + query)
        
        while not response:
            time.sleep(0.1)
    
        rate = response.json()[exchange]
        amount_after_conversion = round(amount * rate, 2)
        
        if amount_after_conversion > self._balance:
            raise ValueError("Insufficient funds")
            
        self._balance -= amount_after_conversion
        self.history.append('Withdraw after conversion of an amount of {} '.format(amount) 
                      + currency 
                      + ' to {} $ at date {} : {}'.format(amount_after_conversion, datetime.datetime.now().date(), description))
        self.save()
        
    def transfert(self, beneficiairy, amount, description):
        if isinstance(amount, (int, float)) == False:
            raise TypeError("The inserted amount is not numeric")
        if amount > self._balance:
            raise ValueError("Insufficient funds")
        self.withdraw(amount, 'Transfert to {} : {}'.format(beneficiairy.name, description))
        beneficiairy.deposit(amount, 'Transfert from {} : {}'.format(self.name, description))
        self.save()
        beneficiairy.save()
        
    def save(self):
        data = {}
        data['name'] = self.name
        data['balance'] = round(self.balance, 2)
        data['history'] = self.history
        with open('account_of_'+ self.name +'.json', 'w') as json_file:
            json.dump(data, json_file)

    @property
    def balance(self):
        return self._balance

    def __repr__(self):
        return '{0.__class__.__name__}(name={0.name}, balance={0.balance})'.format(self)

    def __str__(self):
        history_list = '\n'
        for transaction in self.history:
            history_list += ('\t- ' + transaction + '\n')
        return '\n\tBank account of {} : \n\tCurrent balance : {} \n\tHistory : {} \n'.format(self.name, self.balance, history_list)

In [ ]:
# start of the program

ch = ''
name = ''
currencies = [
    "EUR - Euro",
    "GBP - Great Britain pound (Sterling) (nicknamed Cable)",
    "JPY - Japanese yen",
    "CHF - Swiss Franc (nicknamed Swissie)",
    "AUD - Australian dollar (nicknamed Aussie)",
    "CAD - Canadian dollar (nicknamed Loonie)",
    "CNY - China Yuan Renminbi",
    "NZD - New Zealand dollar (nicknamed Kiwi)",
    "INR - Indian rupee",
    "BZR - Brazilian Real",
    "SEK - Swedish Krona",
    "ZAR - South African Rand",
    "HKD - Hong Kong Dollar"
]

while ch != 7:
    if ch == '1':
        name = str(input("\tEnter your name : "))
        file = Path("./account_of_"+ name +".json")
        if file.is_file():
            print('This account already exists.')
        else:
            account = BankAccount(name)
            print('\n\tAccount {} created successfully.'.format(name))
    elif ch == '2': # Deposit
        name = str(input("\tEnter your name : "))
        file = Path("./account_of_"+ name +".json")
        if file.is_file():
            account = BankAccount(name)
            amount = float(input("\tEnter the amount to deposit : "))
            description = str(input("\tEnter a description for your deposit : "))
            account.deposit(amount, description)
        else:
            print('This account doesn\'t exist')
    elif ch == '3': # Withdraw
        name = str(input("\tEnter your name : "))
        file = Path("./account_of_"+ name +".json")
        if file.is_file():
            account = BankAccount(name)
            amount = float(input("\tEnter the amount to withdraw : "))
            description = str(input("\tEnter a description for your withdraw : "))
            account.withdraw(amount, description)
        else:
            print('This account doesn\'t exist')
    elif ch == '4': # Convert and Withdraw
        name = str(input("\tEnter your name : "))
        file = Path("./account_of_"+ name +".json")
        if file.is_file():
            account = BankAccount(name)
            currency_choice = 0
            while currency_choice < 1 or currency_choice > 13:
                print("\tPlease choose one of the following currencies :")
                for index, cur in enumerate(currencies):
                    print("\t{} - {}".format(index + 1, cur))
                currency_choice = int(input("\tEnter the currency in which you want to do the withdraw : "))
            
            currency = currencies[currency_choice - 1][:3]
            amount = float(input("\tEnter the amount to withdraw : "))
            description = str(input("\tEnter a description for your withdraw : "))
            account.convert_then_withdraw(amount, currency, description)
        else:
            print('This account doesn\'t exist')
    elif ch == '5': # Transfert amount
        name = str(input("\tEnter your name : "))
        file = Path("./account_of_"+ name +".json")
        if file.is_file():
            account = BankAccount(name)
            beneciciary_name = str(input("\tEnter beneciciary's name : "))
            file = Path("./account_of_"+ beneciciary_name +".json")
            if file.is_file():
                beneciciary_account = BankAccount(beneciciary_name)
                amount = float(input("\tEnter the amount to transfert : "))
                description = str(input("\tEnter a description for your transfert : "))
                account.transfert(beneciciary_account, amount, description)
            else:
                print('The account of beneficiary doesn\'t exist')
        else:
            print('This account doesn\'t exist')
    elif ch == '6':
        name = str(input("\tEnter your name : "))
        file = Path("./account_of_"+ name +".json")
        if file.is_file():
            account = BankAccount(name)
            print(account)
        else:
            print('This account doesn\'t exist')
    elif ch == '7':
        print("\tThanks for using Bank Account Managemnt System")
        break
    else :
        if ch is not '':
            print("Invalid choice")
    
    #system("cls");
    print("\tMAIN MENU")
    print("\t1. NEW ACCOUNT")
    print("\t2. DEPOSIT AMOUNT")
    print("\t3. WITHDRAW AMOUNT")
    print("\t4. WITHDRAW AMOUNT IN A SPECIFIC CURRENCY")
    print("\t5. TRANSFERT AMOUNT TO ACCOUNT")
    print("\t6. BALANCE ENQUIRY")
    print("\t7. EXIT")
    print("\tPlease Select Your Option (1-7)")
    #system("cls");

    ch = input("\n\tEnter your choice : ")

	MAIN MENU
	1. NEW ACCOUNT
	2. DEPOSIT AMOUNT
	3. WITHDRAW AMOUNT
	4. WITHDRAW AMOUNT IN A SPECIFIC CURRENCY
	5. TRANSFERT AMOUNT TO ACCOUNT
	6. BALANCE ENQUIRY
	7. EXIT
	Please Select Your Option (1-7)

	Enter your choice : 4
	Enter your name : Alex
	Please choose one of the following currencies :
	1 - EUR - Euro
	2 - GBP - Great Britain pound (Sterling) (nicknamed Cable)
	3 - JPY - Japanese yen
	4 - CHF - Swiss Franc (nicknamed Swissie)
	5 - AUD - Australian dollar (nicknamed Aussie)
	6 - CAD - Canadian dollar (nicknamed Loonie)
	7 - CNY - China Yuan Renminbi
	8 - NZD - New Zealand dollar (nicknamed Kiwi)
	9 - INR - Indian rupee
	10 - BZR - Brazilian Real
	11 - SEK - Swedish Krona
	12 - ZAR - South African Rand
	13 - HKD - Hong Kong Dollar
	Enter the currency in which you want to do the withdraw : 3
	Enter the amount to withdraw : 127
	Enter a description for your withdraw : few yens
	MAIN MENU
	1. NEW ACCOUNT
	2. DEPOSIT AMOUNT
	3. WITHDRAW AMOUNT
	4. WITHDRAW AMOUNT IN A SPECIFIC C

In [39]:
# Testing BankAccount Class
customer1 = BankAccount('Alex')
customer1.deposit(10000, 'depot')
customer1.withdraw(30, 'Description d\'un retrait')
print(customer1)
customer1.convert_then_withdraw(10, 'EUR', 'Description d\'un retrait post conversion')
print(customer1)
customer2 = BankAccount('Sam', 2000)
print(customer2)

customer1.transfert(customer2, 200, 'Pour les 12 grammes d\'hier soir')
print(customer2)


	Bank account of Alex : 
	Current balance : 19728.72 
	History : 
	- Income of an amount of 10000$ at date 2018-11-28 : depot
	- Withdraw of an amount of 30$ at date 2018-11-28 : Description d'un retrait
	- Withdraw after conversion of an amount of 10 EUR to 11.28 $ at date 2018-11-28 : Description d'un retrait post conversion
	- Withdraw of an amount of 200$ at date 2018-11-28 : Transfert of an amount of 200$ at date 2018-11-28 to Sam : Pour les 12 grammes d'hier soir
	- Income of an amount of 10000$ at date 2018-11-28 : depot
	- Withdraw of an amount of 30$ at date 2018-11-28 : Description d'un retrait
 


	Bank account of Alex : 
	Current balance : 19717.440000000002 
	History : 
	- Income of an amount of 10000$ at date 2018-11-28 : depot
	- Withdraw of an amount of 30$ at date 2018-11-28 : Description d'un retrait
	- Withdraw after conversion of an amount of 10 EUR to 11.28 $ at date 2018-11-28 : Description d'un retrait post conversion
	- Withdraw of an amount of 200$ at date 201

In [51]:
currencies = [
    "EUR - Euro",
    "GBP - Great Britain pound (Sterling) (nicknamed Cable)",
    "JPY - Japanese yen",
    "CHF - Swiss Franc (nicknamed Swissie)",
    "AUD - Australian dollar (nicknamed Aussie)",
    "CAD - Canadian dollar (nicknamed Loonie)",
    "CNY - China Yuan Renminbi",
    "NZD - New Zealand dollar (nicknamed Kiwi)",
    "INR - Indian rupee",
    "BZR - Brazilian Real",
    "SEK - Swedish Krona",
    "ZAR - South African Rand",
    "HKD - Hong Kong Dollar"
]


0 EUR - Euro
1 GBP - Great Britain pound (Sterling) (nicknamed Cable)
2 JPY - Japanese yen
3 CHF - Swiss Franc (nicknamed Swissie)
4 AUD - Australian dollar (nicknamed Aussie)
5 CAD - Canadian dollar (nicknamed Loonie)
6 CNY - China Yuan Renminbi
7 NZD - New Zealand dollar (nicknamed Kiwi)
8 INR - Indian rupee
9 BZR - Brazilian Real
10 SEK - Swedish Krona
11 ZAR - South African Rand
12 HKD - Hong Kong Dollar
